### Word Group In Youtube Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
shift = 0.3  # sentence time shift

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")


6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [6]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [7]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [8]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [9]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [10]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [11]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [12]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [13]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [14]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [15]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [18]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [19]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,33.4,"kamyon, araçlar, normalde, kullanılıyor, özgür...","büyük, daha, için"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"bağlamak, kolay, olmayacak, arka, tarafı, burcu",
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"arabaya, yarım, attıracağım, özgür, tur",
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"oldu, biraz, daha, et, tamam, devam"
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,22.3,"kar, şimdilik, burcu, artarsa, tekere, takacağ...","ama, iki"
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"önerilen, listesi, üst, videonun, linkini, köş...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,9.2,"videoda, uyarla, dilediğiniz, bunlardan, ekled...",bu
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,20.1,"videoda, dek, youtube, eğitim, farklı, ücretsi...","olan, bir, lütfen"
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,27.4,"kanalıma, hoşçakalın, bakın, abone, beğenmeyi,...","olarak, çok, iyi"


In [20]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,33.4,"kamyon, araçlar, normalde, kullanılıyor, özgür...","büyük, daha, için"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"bağlamak, kolay, olmayacak, arka, tarafı, burcu",NaN
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"arabaya, yarım, attıracağım, özgür, tur",NaN
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"oldu, biraz, daha, et, tamam, devam"
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,22.3,"kar, şimdilik, burcu, artarsa, tekere, takacağ...","ama, iki"
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"önerilen, listesi, üst, videonun, linkini, köş...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,9.2,"videoda, uyarla, dilediğiniz, bunlardan, ekled...",bu
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,20.1,"videoda, dek, youtube, eğitim, farklı, ücretsi...","olan, bir, lütfen"
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,27.4,"kanalıma, hoşçakalın, bakın, abone, beğenmeyi,...","olarak, çok, iyi"


In [21]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"oldu, biraz, daha, et, tamam, devam"
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, durumu, göstereyim","size, çünkü, şu"
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"şekilde, bir, dakika, bu"
3,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"eder, nehircim, özgür","misin, bana, yardım"
4,20,379.391,381.437,burcu çok güzel görünüyor,8V9tq1pe8eI,50.1,"burcu, görünüyor","güzel, çok"
...,...,...,...,...,...,...,...,...
989483,3063137,499.645,502.335,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8,55.7,"anda, hızlanacak, şeye, dediğim","bir, gibi, çok, her, çünkü"
989484,3063156,65.960,74.150,sonra,MvWp9pWLihA,100.1,NaN,sonra
989485,3063159,91.970,98.270,birlikte toplam,MvWp9pWLihA,50.1,toplam,birlikte
989486,3063191,503.190,513.210,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA,50.1,"birinci, yanındaki, altındaki","ve, hemen, iki, üç, bu"


In [40]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for i in max(all_index_list, key=len):
        word = words[i]
        text_list.append(word)
    text = " ".join(text_list)
    
    return text 

In [42]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [41]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"oldu, biraz, daha, et, tamam, devam",biraz daha devam et devam et tamam oldu
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, durumu, göstereyim","size, çünkü, şu",çünkü
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"şekilde, bir, dakika, bu",bu şekilde bir dakika
3,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"eder, nehircim, özgür","misin, bana, yardım",bana yardım
4,20,379.391,381.437,burcu çok güzel görünüyor,8V9tq1pe8eI,50.1,"burcu, görünüyor","güzel, çok",çok güzel
...,...,...,...,...,...,...,...,...,...
989483,3063137,499.645,502.335,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8,55.7,"anda, hızlanacak, şeye, dediğim","bir, gibi, çok, her, çünkü",gibi her
989484,3063156,65.960,74.150,sonra,MvWp9pWLihA,100.1,NaN,sonra,sonra
989485,3063159,91.970,98.270,birlikte toplam,MvWp9pWLihA,50.1,toplam,birlikte,birlikte
989486,3063191,503.190,513.210,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA,50.1,"birinci, yanındaki, altındaki","ve, hemen, iki, üç, bu",iki ve hemen


In [43]:
df_word_group_time_loc_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.690889,201.338000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,248.832000,249.465081,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.325000,256.234111,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,309.970405,311.117568,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,379.800200,380.700440,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
989483,gibi her,501.121863,501.649314,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8
989484,sonra,65.960000,74.150000,sonra,MvWp9pWLihA
989485,birlikte,91.970000,95.750000,birlikte toplam,MvWp9pWLihA
989486,iki ve hemen,508.473273,511.023818,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA


In [44]:
df_word_group_time_loc_result["start_time"] = df_word_group_time_loc_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_result["end_time"] = df_word_group_time_loc_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.390889,201.638000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,248.532000,249.765081,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.025000,256.534111,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,309.670405,311.417568,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,379.500200,381.000440,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
989483,gibi her,500.821863,501.949314,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8
989484,sonra,65.660000,74.450000,sonra,MvWp9pWLihA
989485,birlikte,91.670000,96.050000,birlikte toplam,MvWp9pWLihA
989486,iki ve hemen,508.173273,511.323818,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA


In [45]:
df_word_group_time_loc_result["start_time"] = df_word_group_time_loc_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_result["end_time"] = df_word_group_time_loc_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197,202,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,249,250,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254,257,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,310,311,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,380,381,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
989483,gibi her,501,502,çok hızlanacak çünkü dediğim gibi her şeye bir...,7HrxKm4FDw8
989484,sonra,66,74,sonra,MvWp9pWLihA
989485,birlikte,92,96,birlikte toplam,MvWp9pWLihA
989486,iki ve hemen,508,511,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA


In [22]:
text = "burcu biraz daha devam et devam et istanbul burcu burcu tamam oldu et izmir devam et devam et burcu burcu daha devam et burcu burcu"

In [23]:
words = re.findall(r"\w+", text, re.UNICODE)
words

['burcu',
 'biraz',
 'daha',
 'devam',
 'et',
 'devam',
 'et',
 'istanbul',
 'burcu',
 'burcu',
 'tamam',
 'oldu',
 'et',
 'izmir',
 'devam',
 'et',
 'devam',
 'et',
 'burcu',
 'burcu',
 'daha',
 'devam',
 'et',
 'burcu',
 'burcu']

In [24]:
len(words)

25

In [25]:
words[1] in word_list

True

In [26]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [27]:
index_list

[1, 2, 3, 4, 5, 6, 10, 11, 12, 14, 15, 16, 17, 20, 21, 22]

In [28]:
index_list[0] 

1

In [29]:
len(index_list)

16

In [30]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [31]:
all_index_list

[[1, 2, 3, 4, 5, 6], [10, 11, 12], [14, 15, 16, 17], [20, 21, 22]]

In [32]:
all_index_list[0]

[1, 2, 3, 4, 5, 6]

In [33]:
max(all_index_list, key=len)

[1, 2, 3, 4, 5, 6]

In [34]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text

'biraz daha devam et devam et'

In [35]:
def exract_word_group(text, word_list):
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for i in max(all_index_list, key=len):
        word = words[i]
        text_list.append(word)
    text = " ".join(text_list)
    
    return text    

In [36]:
text = "burcu biraz daha devam et devam et istanbul burcu burcu tamam oldu et izmir devam et devam et burcu burcu daha devam et burcu burcu"

In [37]:
exract_word_group(text, word_list)

'biraz daha devam et devam et'

In [38]:
df_adjust_sentence_ratio_test = df_adjust_sentence_ratio.head(100)

In [39]:
df_adjust_sentence_ratio_test.loc[:,"search_string"] = df_adjust_sentence_ratio_test.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio_test

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"oldu, biraz, daha, et, tamam, devam",biraz daha devam et devam et tamam oldu
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, durumu, göstereyim","size, çünkü, şu",çünkü
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"şekilde, bir, dakika, bu",bu şekilde bir dakika
3,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"eder, nehircim, özgür","misin, bana, yardım",bana yardım
4,20,379.391,381.437,burcu çok güzel görünüyor,8V9tq1pe8eI,50.1,"burcu, görünüyor","güzel, çok",çok güzel
...,...,...,...,...,...,...,...,...,...
95,371,30.340,35.540,sen orada yine öyle tek başına,qnyDewp1vTE,83.4,başına,"öyle, yine, sen, orada, tek",sen orada yine öyle tek
96,377,59.220,61.780,yine üşüyorsun,qnyDewp1vTE,50.1,üşüyorsun,yine,yine
97,378,61.780,64.040,hissetmiyor musun,qnyDewp1vTE,50.1,hissetmiyor,musun,musun
98,379,64.380,67.360,yine,qnyDewp1vTE,100.1,NaN,yine,yine


In [227]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [228]:
df_word_group_time_loc_result = word_group_time_loc(df_adjust_sentence_ratio_test, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.555556,202.000000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,249.000000,249.648649,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.000000,256.444444,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,bana yardım,309.513514,310.918919,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
4,çok güzel,379.600000,380.920000,burcu çok güzel görünüyor,8V9tq1pe8eI
...,...,...,...,...,...
95,sen orada yine öyle tek,30.000000,34.800000,sen orada yine öyle tek başına,qnyDewp1vTE
96,yine,59.000000,60.071429,yine üşüyorsun,qnyDewp1vTE
97,musun,62.941176,64.000000,hissetmiyor musun,qnyDewp1vTE
98,yine,64.000000,68.000000,yine,qnyDewp1vTE
